In [31]:
import os
from pydantic import BaseModel
from pydantic.fields import Field
from openai import OpenAI
from dotenv import load_dotenv
import prompts
import context_prompts
import yaml

In [32]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = OPENAI_API_KEY)

In [33]:
def answer(msg):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a data warehouse designer. I’m the end-user."
            },
            {
                "role": "user", 
                "content": msg
            }
        ]
    )
    return response.choices[0].message.content

In [34]:
def answer_RQ1(msg):
    response = client.chat.completions.create(
        
        model = "gpt-4o-mini",
        temperature = 0,
        messages=[
            {
                "role": "user",
                "content": msg
            }
        ]
    )
    return response.choices[0].message.content

In [35]:
def save_to_file(filename, content):
    """
    Saves content to a file.
    """
    with open(filename, 'w') as file:
        file.write(content)

In [36]:
filename = "input/file.yml"
with open(filename, 'r') as file:
    content = file.read()

In [37]:
content

'fact:\nname: PURCHASE\nmeasures:\n- name: PURCHASE.quantity\n- name: PURCHASE.revenue\n- name: ARTICLES.unitPrice\ndependencies:\n- from: TYPES.typeId\nto: TYPES.typeName\n- from: ARTICLES.artId\nto: ARTICLES.artName\n- from: ARTICLES.artId\nto: TYPES.typeId\nrole: type\n- from: REGIONS.regId\nto: REGIONS.regionName\n- from: REGIONS.regId\nto: REGIONS.state\n- from: REGIONS.state\nto: REGIONS.country\n- from: STORES.storeId\nto: STORES.storeName\n- from: STORES.storeId\nto: STORES.storeAddress\n- from: STORES.storeId\nto: REGIONS.regId\nrole: region\n- from: PURCHASE\nto: PURCHASE.quantity\n- from: PURCHASE\nto: PURCHASE.revenue\n- from: PURCHASE\nto: ARTICLES.unitPrice\n- from: PURCHASE\nto: ARTICLES.artId\n- from: PURCHASE\nto: STORES.storeId\n- from: PURCHASE\nto: PURCHASE.date'

In [39]:
for q_prompt in dir(prompts):

    if q_prompt.startswith("RQ1"):
        prompt_value = getattr(prompts, q_prompt)
        answer_text = answer_RQ1(prompt_value)
        save_to_file(f"output/{q_prompt}.txt", answer_text)


    for context_prompt in dir(context_prompts):
        
        if context_prompt.startswith("__") or q_prompt.startswith("__"):  # Skip built-in attributes
            continue

        # Combine context and question prompts for RQ2
        if context_prompt.endswith("q2") and q_prompt.startswith("RQ2"):
            combined_prompt = getattr(context_prompts, context_prompt).format(
                content=content,
                msg=getattr(prompts, q_prompt)
            )
            answer_text = answer(combined_prompt)
            var_name = f"{context_prompt}_{q_prompt}"
            save_to_file(f"output/{var_name}.yaml", answer_text)

        # Combine context and question prompts for RQ3
        elif context_prompt.endswith("q3") and q_prompt.startswith("RQ3"):
            combined_prompt = getattr(context_prompts, context_prompt).format(
                content=content,
                msg=getattr(prompts, q_prompt)
            )
            answer_text = answer(combined_prompt)
            var_name = f"{context_prompt}_{q_prompt}"
            save_to_file(f"output/{var_name}.yaml", answer_text)